# Imperiled Aquatic Species

This is a draft indicator for the 2023 blueprint minor update.

This is a proposed change to the way we handle 0 vs nodata in our indicators.

This must be run in ArcPro with an open map because of the join.

Created by Amy Keister, last run by Amy Keister on 15 May, 2023. It took 40 minutes to run.

In [27]:
import os
import arcpy

In [28]:
import time
start = time.time()

In [29]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestVirginiaImp\WestVir.gdb"

In [30]:
# define final outputs
out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestVirginiaImp\WestVirginiaImperiledAquaticSpecies.tif"

In [31]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [32]:
# define inputs
#SARP = r"F:\GIS_DATA\WaterResources\SARP2023\spp_HUC12.xlsx"
#SARP = r"F:\GIS_DATA\WaterResources\SARP2023\spp_HUC12.xlsx\Sheet1$"
Env = r"F:\GIS_DATA\SpeciesAndHabitats\EPAEnviroAtlas\CONUS_metrics_Sep2021_FGDB\CONUS_metrics_Sep2021_FGDB.gdb\natureserve_NHDPv2_WBD"
FP = r"F:\GIS_DATA\WaterResources\Estimated_floodplain_CONUS\Estimated_floodplain_CONUS.tif"
H12 = r"F:\GIS_DATA\WaterResources\NHD\WBD_National_GDB\WBD_National_GDB.gdb\WBDHU12"
state= r"F:\GIS_DATA\Boundaries\States\tl_rd22_us_state\tl_rd22_us_state.shp"
catch = r"F:\GIS_DATA\WaterResources\NHD\NHDPlusV21\NHDPlusV21_Catchment.gdb\Catchment"
# can't use high resolution for this purpose because the NC coast is messed up
#catch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NHDPlusCatchment"

### Start Analysis

In [33]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [34]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestVirginiaImp\WestVir.gdb


### SARP Data


In [18]:
# make a copy of the HUC12 for edits
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.CopyFeatures(H12, "H12", '', None, None, None)

In [19]:
# join table from Enviro Atlas with HUC12
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.AddJoin("H12", "huc12", Env, "HUC_12", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")

In [20]:
# convert to raster using the "AQ_TOT_A" field
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster("H12", "natureserve_NHDPv2_WBD.AQ_TOT_A", "AqTotA", "CELL_CENTER", "NONE", 30)

In [21]:
# reclassify
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Reclassify("AqTotA", "Value", "NODATA 1;0 1;1 2;2 3;3 4;4 100 5", "DATA"); out_raster.save("AqTotARclss")

In [22]:
# Limit the aquatic data to the EPA floodplain
out_raster = arcpy.sa.Con(FP, "AqTotARclss", 0,"Value = 1"); out_raster.save("imperiled")

### Clip to medium resolution catchments

In [23]:
# turn NHD medium resolution catchments to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster(catch, "GRIDCODE", "catchr", "CELL_CENTER", "NONE", Rextent)

In [24]:
# The EPA floodplain goes too far into the estuaries and oceans, we need to clip it back. We are using NHDPlus medium resolution 
# catchments for this
out_raster = arcpy.sa.Con("catchr", "imperiled", '',"Value > 1"); out_raster.save("imperiledA")

### Since these data are only used for West Virginia entirely, we need to remove everythging that isn't West Virginia

In [25]:
# pull out west virgina from the state boundary file 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.analysis.Select(state, "WV", "STUSPS = 'WV'")

In [26]:
# add field for raster conversion
arcpy.management.CalculateField('WV', "raster", "1", "PYTHON3", "", "SHORT")

<Result 'WV'>

In [27]:
# convert West Virginia polygon to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster('WV', "raster", "WVr", "CELL_CENTER", "NONE", Rextent)

In [28]:
# make all values in West Virginia nodata
out_raster = arcpy.sa.Times("imperiledA", "WVr"); out_raster.save('imperiledAq')

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [29]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 5:
        return '5 = 4+ aquatic imperiled (G1/G2) or threatened/endangered animal species observed'
    elif value == 4:
        return '4 = 3 aquatic imperiled (G1/G2) or threatened/endangered animal species observed'
    elif value == 3:
        return '3 = 2 aquatic imperiled (G1/G2) or threatened/endangered animal species observed'
    elif value == 2:
        return '2 = 1 aquatic imperiled (G1/G2) or threatened/endangered animal species observed'
    elif value == 1:
        return '1 = 0 aquatic imperiled (G1/G2) or threatened/endangered animal species observed'
    elif value == 0:
        return '0 = Not identified as a floodplain' 
"""

In [30]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("imperiledAq", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestVirginiaImp\\WestVir.gdb\\imperiledAq'>

In [35]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 9:
		return 8
	if Value == 8:
		return 36
	if Value == 7:
		return 32
	if Value == 6:
		return 25
	if Value == 5:
		return 57
	if Value == 4:
		return 115
	if Value == 3:
		return 187
	if Value == 2:
		return 233
	if Value == 1:
		return 255
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 9:
		return 29
	if Value == 8:
		return 51
	if Value == 7:
		return 93
	if Value == 6:
		return 144
	if Value == 5:
		return 181
	if Value == 4:
		return 205
	if Value == 3:
		return 233
	if Value == 2:
		return 248
	if Value == 1:
		return 255
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 9:
		return 88
	if Value == 8:
		return 148
	if Value == 7:
		return 168
	if Value == 6:
		return 192
	if Value == 5:
		return 196
	if Value == 4:
		return 184
	if Value == 3:
		return 161
	if Value == 2:
		return 152
	if Value == 1:
		return 192
	if Value == 0:
		return 255
	else:
		return 255
"""

In [36]:
# calculate Red field
arcpy.management.CalculateField("imperiledAq", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("imperiledAq", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("imperiledAq", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'imperiledAq'>

In [37]:
# clip to base Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("imperiledAq", Rextent); out_raster.save("mask")

In [38]:
# export as .tif with base extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("mask", out, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [35]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

2028.027860403061
